# Data Preparation

Match data is not normalized at the moment, i.e. 
- several events appear in one row
- extra time, penalty and golden goal all appear in win conditions

This notebook reads raw data and saves results to csv

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools

df_matches=pd.read_csv('data_raw/WorldCupMatches.csv', sep=',')
df_players=pd.read_csv('data_raw/WorldCupPlayers.csv', sep=',')
df_cups=pd.read_csv('data_raw/WorldCups.csv', sep=',')

df_joined_matches = pd.merge(df_matches, df_players, on=['RoundID', 'MatchID'])

In [11]:
#G=Goal, OG=Own Goal, Y=Yellow Card, R=Red Card, SY = Red Card by second yellow, P=Penalty, MP=Missed Penalty, I = Substitution In, O=Substitute Out, IH= In half time?
df_joined_matches = df_joined_matches.replace(np.nan, '', regex=True)
# explode rows with multiple events
df_joined_matches_exploded_series = pd.DataFrame(df_joined_matches.Event.str.split(' ').tolist()).stack()
# join
df_joined_matches_exploded = pd.merge(df_joined_matches.reset_index(), df_joined_matches_exploded_series.to_frame().reset_index(), how = 'left', left_on = 'index', right_on = 'level_0')
# drop useless columns
df_joined_matches_exploded.drop(['Event','level_0','level_1','index'], 1, inplace=True)
df_joined_matches_exploded.columns.values[26] = 'Event'
df_joined_matches_exploded

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,...,MatchID,Home Team Initials,Away Team Initials,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,FRA,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,
1,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,MEX,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,
2,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,FRA,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,,G40'
3,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,MEX,LUQUE Juan (MEX),S,0,Juan CARRENO,,G70'
4,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,FRA,CAUDRON Raoul (FRA),S,0,Ernest LIBERATI,,
5,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,MEX,LUQUE Juan (MEX),S,0,Rafael GARZA,C,
6,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,FRA,CAUDRON Raoul (FRA),S,0,Andre MASCHINOT,,G43'
7,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,FRA,CAUDRON Raoul (FRA),S,0,Andre MASCHINOT,,G87'
8,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,MEX,LUQUE Juan (MEX),S,0,Hilario LOPEZ,,
9,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,1096,FRA,MEX,FRA,CAUDRON Raoul (FRA),S,0,Etienne MATTLER,,


In [12]:
def filter_event(x):
    return "".join(itertools.takewhile(str.isalpha, x.Event))

def filter_minute(x):
    return ''.join(filter(lambda x: x.isdigit(), x.Event))

# Filter out EventType and Minute
df_joined_matches_exploded['EventType'] = df_joined_matches_exploded.apply(filter_event, axis=1)
df_joined_matches_exploded['EventMinute'] = df_joined_matches_exploded.apply(filter_minute, axis=1)
df_joined_matches_exploded.drop(['Event'], 1,inplace=True)

df_joined_matches_exploded.rename(columns={'Win conditions': 'WinConditions'}, inplace=True)

In [13]:
df_joined_matches_exploded = df_joined_matches_exploded.assign(ExtraTime = lambda x: x.WinConditions.str.contains("extra time"))
df_joined_matches_exploded = df_joined_matches_exploded.assign(Penalty = lambda x: x.WinConditions.str.contains("penalties"))
df_joined_matches_exploded = df_joined_matches_exploded.assign(GoldenGoal = lambda x: x.WinConditions.str.contains("Golden Goal"))
# Obviously Golden Goal means extra time
df_joined_matches_exploded.loc[df_joined_matches_exploded.GoldenGoal == True, 'ExtraTime'] = True
df_joined_matches_exploded.loc[df_joined_matches_exploded.Penalty == True, 'ExtraTime'] = True
df_joined_matches_exploded

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,WinConditions,...,Coach Name,Line-up,Shirt Number,Player Name,Position,EventType,EventMinute,ExtraTime,Penalty,GoldenGoal
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,,,False,False,False
1,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,,,False,False,False
2,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,,G,40,False,False,False
3,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,LUQUE Juan (MEX),S,0,Juan CARRENO,,G,70,False,False,False
4,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,CAUDRON Raoul (FRA),S,0,Ernest LIBERATI,,,,False,False,False
5,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,LUQUE Juan (MEX),S,0,Rafael GARZA,C,,,False,False,False
6,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,CAUDRON Raoul (FRA),S,0,Andre MASCHINOT,,G,43,False,False,False
7,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,CAUDRON Raoul (FRA),S,0,Andre MASCHINOT,,G,87,False,False,False
8,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,LUQUE Juan (MEX),S,0,Hilario LOPEZ,,,,False,False,False
9,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,CAUDRON Raoul (FRA),S,0,Etienne MATTLER,,,,False,False,False


In [14]:
def transform_penalty_away(x):
    if(x.Penalty):
        return int(x.WinConditions[-3])
    else:
        return 0
    
def transform_penalty_home(x):
    if(x.Penalty):
        return int(x.WinConditions[-7])
    else:
        return 0
    
df_joined_matches_exploded['DecisionPenaltyAway'] = df_joined_matches_exploded.apply(transform_penalty_away, axis=1)
df_joined_matches_exploded['DecisionPenaltyHome'] = df_joined_matches_exploded.apply(transform_penalty_home, axis=1)

df_joined_matches_exploded.drop(['WinConditions'], 1,inplace=True)

In [15]:
df_joined_matches_exploded = df_joined_matches_exploded.assign(HourGameStart = lambda x: x.Datetime.str[-6:-4])

In [16]:
df_joined_matches_exploded = df_joined_matches_exploded.replace(np.nan, '', regex=True)
df_joined_matches_exploded

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Attendance,...,Player Name,Position,EventType,EventMinute,ExtraTime,Penalty,GoldenGoal,DecisionPenaltyAway,DecisionPenaltyHome,HourGameStart
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Alex THEPOT,GK,,,False,False,False,0,0,15
1,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Oscar BONFIGLIO,GK,,,False,False,False,0,0,15
2,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Marcel LANGILLER,,G,40,False,False,False,0,0,15
3,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Juan CARRENO,,G,70,False,False,False,0,0,15
4,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Ernest LIBERATI,,,,False,False,False,0,0,15
5,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Rafael GARZA,C,,,False,False,False,0,0,15
6,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Andre MASCHINOT,,G,43,False,False,False,0,0,15
7,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Andre MASCHINOT,,G,87,False,False,False,0,0,15
8,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Hilario LOPEZ,,,,False,False,False,0,0,15
9,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,4444,...,Etienne MATTLER,,,,False,False,False,0,0,15


In [19]:
df_joined_matches_exploded.Stage.unique()

array(['Group 1', 'Group 4', 'Group 2', 'Group 3', 'Semi-finals', 'Final',
       'Preliminary round', 'Quarter-finals', 'Match for third place',
       'First round', 'Group 6', 'Group B', 'Group A', 'Group 5',
       'Group C', 'Group D', 'Group F', 'Group E', 'Round of 16',
       'Group H', 'Group G', 'Third place', 'Play-off for third place'],
      dtype=object)

In [20]:
def compute_stage(x):
    if("Preliminary" in x['Stage']):
        return 0
    if("Group" in x['Stage']):
        return 1
    if("Round of 16" in x['Stage']):
        return 2
    if("First round" in x['Stage']):
        return 2
    if("Quarter-finals" in x['Stage']):
        return 3
    if("Semi-finals" in x['Stage']):
        return 4
    if("Play-off for third place" in x['Stage']):
        return 4
    if("Match for third place" in x['Stage']):
        return 4
    if("Third place" in x['Stage']):
        return 5
    if("Final" in x['Stage']):
        return 6
    else:
        return 99

df_joined_matches_exploded["StageRank"] = df_joined_matches_exploded.apply(lambda row: compute_stage(row), axis=1)

In [21]:
df_joined_matches_exploded.to_csv("data_prepared/event.csv",index=False)